#%% md

# Instituto de Educação Superior de Brası́lia – IESB
## Pós-Graduação em Inteligência Artificial
### Deep Learning e Redes Neurais

### Atividade 1 - Classificação de imagens na base de dados CIFAR-10 utilizando MLP


#### Descrição do Dataset CIFAR-10

O dataset CIFAR-10 consiste de 60 mil imagens coloridas, com 10 classes distintas igualmente balanceadas (isto é, 6 mil imagens por classe).

O conjunto é separado em 50 mil imagens para treinamento e 10 mil para validação (teste).

As imagens são de animais e objetos, com as seguintes classes:


  - 0 - Avião										
  - 1 - Automóvel
  - 2 - Pássaro
  - 3 - Gato
  - 4 - Cervo
  - 5 - Cachorro
  - 6 - Sapo
  - 7 - Cavalo
  - 8 - Barco
  - 9 - Caminhão

Fonte: https://www.cs.toronto.edu/~kriz/cifar.html

## 1) Dataset

 - Carregue o dataset.
```python
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
```

 - Escolha, aletoriamente (e de forma automatizada), 16 imagens na base de treino, e apresente-as visualmente em um gráfico 4x4 (subplot).
 
 
 - Verifique se as dimensões dos tensores estão de acordo com a descrição do dataset.


 - Realize, caso necessário, pré-processamento nos dados. (Ex: normalização, padronização, codificação de classes, etc).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def visualizar_img (class_name):
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.show()

In [ ]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

nb_epoch = 200
batch_size = 128
num_classes = 10

class_names = ['Avião', 'Automóvel', 'Pássaro', 'Gato', 'Cervo', 'Cachorro', 'Sapo', 'Cavalo', 'Barco', 'Caminhão']

visualizar_img(class_names)


In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
import tensorflow as tf
n_classes = 10

X_train = X_train.reshape(50000, 32 * 32 * 3)
X_test = X_test.reshape(10000, 32 * 32 * 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
y_train

In [ ]:
X_train

## 2) Implementação do Modelo MLP

### 2.1 Arquitetura

 - Defina uma arquitera MLP utilzando a API Sequencial do Keras.
 
 
 - A escolha de parâmetros do modelo, como, por exemplo, *quantidade de camadas ocultas*, *quantidade de neurônios*, *funções de ativação*, fica a critério do projetista.
 
 
 - Utilize métodos de reguralização para evitar *overfitting*.
 
 
 - Apresente um resumo da arquitetura implementada

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

num_classes = 10

model = Sequential()
model.add(Dense(1024, input_shape=(3072, )))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()


### 2.2 Compilação

 - Baseado no problema descrito, defina a função custo que será otimizada, e qual otimizador será utilizado.
 
 
 - Defina também quais métricas serão avaliadas pelo modelo durante a etapa de treinamento e de validação.


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])

### 2.3 Treinamento e Validação
 - Como as bases de treino e de teste já estão separadas de origem, essas etapas podem ser realizadas em conjunto. 
   (Consulte a documentação do método *fit* para saber como fazer.)
 
 
 - Defina a quantidade máxima de épocas para o treinamento.
 
 
 - Utilize rotinas de *callback* para que o treinamento não ocorra indefinidamente até atingir o número máximo de épocas definido desnecessariamente.
 
 
 - Salve o melhor modelo treinado.
 
 
 - Mostre em um gráfico o comportamento da função custo (*loss*) e da acurácia, ao longo das epócas, para as bases de treino e de validação.
 
 
 - O modelo deve atingir um mínimo de 50% de acurácia média na base de validação. Caso não atinja essa meta, revise a arquitetura definida.


In [ ]:
cb = [ tf.keras.callbacks.EarlyStopping(monitor='val_loss',
           patience=500,
           restore_best_weights=True)
       ]

print('treinamento...')

# training
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, y_test))

#history = model.fit(X_train, 
#          y_train, 
#          epochs=2000, 
#          callbacks=cb,
#          verbose=2,
#          validation_data=(X_test, y_test),
#          )
print(history)
    

In [ ]:

import matplotlib.pyplot as plt

print(history.history.keys())

plt.plot(history.history['accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 2.4 Desempenho na Base de Teste

 - Realize, com o modelo treinado, predições na base de teste (método *predict*).
 
 - Com as predições, calcule a precisão e a revocação para cada classe. (Pode ser utilizada a função *classification_report* da o *scikit-learn* ou alguma outra similar).


In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)

print('Test loss:', loss)
print('Test acc:', acc)

In [ ]:

predict = model.predict(X_test)

predict_classes = model.predict_classes(X_test)

predict, predict_classes

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

from sklearn.metrics import classification_report

print(classification_report(y_test, predict_classes))


